In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/m5-pointforecast-itstorder-hist-val/_28dl_dict.pkl
/kaggle/input/m5-pointforecast-itstorder-hist-val/_28dl_14ds_dict.pkl
/kaggle/input/m5-pointforecast-itstorder-hist-val/_28dl_21dm_dict.pkl
/kaggle/input/m5-pointforecast-itstorder-hist-val/_28dl_21ds_dict.pkl
/kaggle/input/m5-pointforecast-itstorder-hist-val/_28dl_7ds_dict.pkl
/kaggle/input/m5-pointforecast-itstorder-hist-val/custom.css
/kaggle/input/m5-pointforecast-itstorder-hist-val/__notebook__.ipynb
/kaggle/input/m5-pointforecast-itstorder-hist-val/hist_feats.pkl
/kaggle/input/m5-pointforecast-itstorder-hist-val/_28dl_14dm_dict.pkl
/kaggle/input/m5-pointforecast-itstorder-hist-val/_28dl_7dm_dict.pkl
/kaggle/input/m5-pointforecast-itstorder-hist-val/_28dl_28dm_dict.pkl
/kaggle/input/m5-pointforecast-itstorder-hist-val/__results__.html
/kaggle/input/m5-pointforecast-itstorder-hist-val/__output__.json
/kaggle/input/m5-pointforecast-itstorder-hist-val/ordered_train_df.pkl
/kaggle/input/m5-pointforecast-itstorder-hist-va

In [2]:
import pickle
from collections import defaultdict
import gc, sys

In [3]:
with open('../input/m5-pointforecast-itstorder-hist-val/ordered_train_df.pkl', 'rb') as f:
    train_df = pickle.load(f)
with open('../input/m5-pointforecast-itstorder-hist-val/ordered_val_df.pkl', 'rb') as f:
    val_df = pickle.load(f)
with open('../input/m5-pointforecast-itstorder-hist-val/val_weights.pkl', 'rb') as f:
    val_weights = pickle.load(f)

In [4]:
train_df.shape, val_df.shape

((14856142, 53), (809284, 53))

In [5]:
class Category:
    def __init__(self, unique_items, item_to_id={}):
        self.items = unique_items
        self.item_to_id = item_to_id
        for _id, item_name in enumerate(self.items):
            self.item_to_id[item_name] = _id
        self.type = np.int16
        if len(self.items) > 30000:
            self.type = np.int32
    
    def encode_series(self, items):
        return np.array([self.item_to_id[_item] for _item in items]).astype(self.type)

In [6]:
with open('../input/m5-point-forecast-createinput-1100/cat_feats.pkl', 'rb') as f:
    cat_feats = pickle.load(f)
with open('../input/m5-point-forecast-createinput-1100/cat_feats_name2idx.pkl', 'rb') as f:
    cat_feats_name2idx = pickle.load(f)
with open('../input/m5-point-forecast-createinput-1100/cat_feats_objs.pkl', 'rb') as f:
    cat_feats_objs = pickle.load(f)

In [7]:
with open('../input/m5-pointforecast-agggroups-idxs-k-val/agg_groups_idxs_K', 'rb') as f:
    agg_groups = pickle.load(f)

In [8]:
mask = (val_weights.weight != 0).values

In [9]:
w = val_weights.weight.values.astype(np.float64)

In [10]:
w = w[mask]

In [11]:
K = val_weights.K.values.astype(np.float64)

In [12]:
K = K[mask]

In [13]:
K.min(), K.max()

(0.014242517733690457, 6.376486059585256)

In [14]:
#sales_mean = val_weights.sales_mean.values[mask]

In [15]:
#sales_std = val_weights.sales_std.values[mask]

In [16]:
num_is = (val_weights.weight != 0).sum(); num_is

28903

In [17]:
import lightgbm as lgb

In [18]:
gc.collect()

20

In [19]:
y_train = train_df.y.values.astype(np.float64)
y_val = val_df.y.values.astype(np.float64)

In [20]:
y_train, y_val

(array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 2., 5., 1.]))

In [21]:
X_train_df = train_df.drop('y', axis=1)
X_val_df = val_df.drop('y', axis=1)

In [22]:
train_dset = lgb.Dataset(X_train_df, y_train)

In [23]:
val_dset = lgb.Dataset(X_val_df, y_val, reference=train_dset)

In [24]:
agg_groups[0]

(array([ 1508,  1509,  1510, ..., 28900, 28901, 28902]), 0.0002545815217077057)

In [36]:
def custom_obj(preds, train_ds):
    #calculate L for each (item,store)
    y = train_ds.get_label()
    yt = y.reshape(num_is, -1)
    predst = preds.reshape(num_is, -1)
    
    L = np.sum(((yt - predst)**2), axis=1)
    L = L[:,None]
    h = float(yt.shape[1])
    _w = 1.0 * (0.0 + 1e2*w)
    _w = _w/12.0
    grads = -1.0 * _w[:,None] * K[:,None] * (yt - predst) * (h**-0.5) * (L**-0.5)
    hess = (-1.0) * K[:,None] * _w[:,None] * (h**-0.5) * ((-1.0)*(L**-0.5) + ((yt - predst)**2)*(L**-1.5))
    # iterate over agg_groups each one is a single time series
    for idxs_g, K_g in agg_groups:
        yt_g = yt[idxs_g,:]
        predst_g = predst[idxs_g,:]
        _w_g = _w[idxs_g].sum()
        L_g = np.sum((yt_g - predst_g)**2)
        grads_g = -1.0 * _w_g * K_g * (yt_g - predst_g) * (h**-0.5) * (L_g**-0.5) 
        grads[idxs_g,:] = grads[idxs_g,:] + grads_g
        hess_g = (-1.0) * K_g * _w_g * (h**-0.5) * ((-1.0)*(L_g**-0.5) + ((yt_g - predst_g)**2)*(L_g**-1.5))
        hess[idxs_g,:] = hess[idxs_g,:] + hess_g    
    grads = grads.reshape(-1)
    hess = hess.reshape(-1)
    return (grads, hess)

In [26]:
def custom_eval(preds, train_ds):
    y = train_ds.get_label()
    yt = y.reshape(num_is, -1)
    predst = preds.reshape(num_is, -1)
    #custom_eval
    h = float(yt.shape[1])
    eval_result = np.sum(w * K * (h**-0.5) * (np.sum(((yt-predst)**2), axis=1)**0.5)) 
    for idxs_g, K_g in agg_groups:
        yt_g = yt[idxs_g,:].sum(axis=0)
        predst_g = predst[idxs_g,:].sum(axis=0)
        _w_g = w[idxs_g].sum()
        eval_result += (_w_g * K_g * (np.sum((yt_g-predst_g)**2)**0.5)/ (h**0.5))
    eval_result = eval_result * (1.0/12.0)
    return ('wrmsse', eval_result, False)

In [27]:
col_names = list(X_train_df.columns.values)

In [28]:
col_names

['item_id',
 'dept_id',
 'cat_id',
 'store_id',
 'state_id',
 'day_id',
 'week_id',
 'year',
 'month',
 'day_of_month',
 'day_of_week',
 'snap',
 'sell_price',
 'Chanukah End_lag',
 'SuperBowl_lag',
 'StPatricksDay_lag',
 'LaborDay_lag',
 'Pesach End_lag',
 'PresidentsDay_lag',
 'NewYear_lag',
 'ColumbusDay_lag',
 'Ramadan starts_lag',
 'Purim End_lag',
 'LentWeek2_lag',
 'Thanksgiving_lag',
 'IndependenceDay_lag',
 'Christmas_lag',
 'NBAFinalsStart_lag',
 "Father's day_lag",
 'LentStart_lag',
 'EidAlAdha_lag',
 'VeteransDay_lag',
 'MartinLutherKingDay_lag',
 'Cinco De Mayo_lag',
 'Halloween_lag',
 'Easter_lag',
 'NBAFinalsEnd_lag',
 'OrthodoxChristmas_lag',
 'ValentinesDay_lag',
 'MemorialDay_lag',
 'Eid al-Fitr_lag',
 'OrthodoxEaster_lag',
 "Mother's day_lag",
 '_28dl',
 '_28dl_7dm',
 '_28dl_7ds',
 '_28dl_14dm',
 '_28dl_14ds',
 '_28dl_21dm',
 '_28dl_21ds',
 '_28dl_28dm',
 '_28dl_28ds']

In [29]:
gc.collect()

80

In [30]:
# weight_scale 1e4
params = {
    'boosting_type': 'gbdt',
    'num_leaves': 512,
    'learning_rate': 0.1,
    'metrics':'l2',
    #'feature_fraction': 0.9,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    'verbose': 1
}

gbm = lgb.train(params, train_dset, valid_sets=[val_dset, train_dset], feature_name=col_names, categorical_feature=[0,1,2,3,4],
                fobj=custom_obj, feval=custom_eval, num_boost_round=300)

/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 1, 2, 3, 4]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[1]	training's l2: 12.9237	training's wrmsse: 4.25773	valid_0's l2: 13.8026	valid_0's wrmsse: 4.85649
[2]	training's l2: 11.5349	training's wrmsse: 3.86867	valid_0's l2: 12.1625	valid_0's wrmsse: 4.41497
[3]	training's l2: 10.3986	training's wrmsse: 3.5199	valid_0's l2: 10.8178	valid_0's wrmsse: 4.01653
[4]	training's l2: 9.47179	training's wrmsse: 3.20696	valid_0's l2: 9.76277	valid_0's wrmsse: 3.65988
[5]	training's l2: 8.71151	training's wrmsse: 2.92638	valid_0's l2: 8.87664	valid_0's wrmsse: 3.34094
[6]	training's l2: 8.08797	training's wrmsse: 2.67495	valid_0's l2: 8.15501	valid_0's wrmsse: 3.05285
[7]	training's l2: 7.57721	training's wrmsse: 2.44984	valid_0's l2: 7.56467	valid_0's wrmsse: 2.79687
[8]	training's l2: 7.15779	training's wrmsse: 2.24723	valid_0's l2: 7.07616	valid_0's wrmsse: 2.56383
[9]	training's l2: 6.80749	training's wrmsse: 2.06612	valid_0's l2: 6.67395	valid_0's wrmsse: 2.35607
[10]	training's l2: 6.51677	training's wrmsse: 1.90392	valid_0's l2: 6.36686	valid_

[80]	training's l2: 4.15673	training's wrmsse: 0.508148	valid_0's l2: 4.72634	valid_0's wrmsse: 0.573489
[81]	training's l2: 4.14774	training's wrmsse: 0.506342	valid_0's l2: 4.7231	valid_0's wrmsse: 0.572519
[82]	training's l2: 4.138	training's wrmsse: 0.505245	valid_0's l2: 4.71754	valid_0's wrmsse: 0.572293
[83]	training's l2: 4.13158	training's wrmsse: 0.50401	valid_0's l2: 4.71551	valid_0's wrmsse: 0.573926
[84]	training's l2: 4.12311	training's wrmsse: 0.501879	valid_0's l2: 4.71278	valid_0's wrmsse: 0.572865
[85]	training's l2: 4.1165	training's wrmsse: 0.500773	valid_0's l2: 4.71134	valid_0's wrmsse: 0.573375
[86]	training's l2: 4.10992	training's wrmsse: 0.500126	valid_0's l2: 4.71032	valid_0's wrmsse: 0.574284
[87]	training's l2: 4.10382	training's wrmsse: 0.498823	valid_0's l2: 4.70979	valid_0's wrmsse: 0.574205
[88]	training's l2: 4.09803	training's wrmsse: 0.497147	valid_0's l2: 4.70889	valid_0's wrmsse: 0.571162
[89]	training's l2: 4.08958	training's wrmsse: 0.495255	vali

[158]	training's l2: 3.74753	training's wrmsse: 0.43866	valid_0's l2: 4.65012	valid_0's wrmsse: 0.550314
[159]	training's l2: 3.74399	training's wrmsse: 0.437807	valid_0's l2: 4.64851	valid_0's wrmsse: 0.549477
[160]	training's l2: 3.73938	training's wrmsse: 0.437252	valid_0's l2: 4.64928	valid_0's wrmsse: 0.549176
[161]	training's l2: 3.73458	training's wrmsse: 0.436646	valid_0's l2: 4.64816	valid_0's wrmsse: 0.548948
[162]	training's l2: 3.72991	training's wrmsse: 0.436013	valid_0's l2: 4.64766	valid_0's wrmsse: 0.549903
[163]	training's l2: 3.72784	training's wrmsse: 0.43554	valid_0's l2: 4.64781	valid_0's wrmsse: 0.549692
[164]	training's l2: 3.72362	training's wrmsse: 0.435245	valid_0's l2: 4.64714	valid_0's wrmsse: 0.549469
[165]	training's l2: 3.72121	training's wrmsse: 0.43508	valid_0's l2: 4.6477	valid_0's wrmsse: 0.549475
[166]	training's l2: 3.71722	training's wrmsse: 0.434129	valid_0's l2: 4.64777	valid_0's wrmsse: 0.548402
[167]	training's l2: 3.7126	training's wrmsse: 0.4

[236]	training's l2: 3.52638	training's wrmsse: 0.411854	valid_0's l2: 4.6361	valid_0's wrmsse: 0.548786
[237]	training's l2: 3.52334	training's wrmsse: 0.411601	valid_0's l2: 4.63654	valid_0's wrmsse: 0.548388
[238]	training's l2: 3.52035	training's wrmsse: 0.41141	valid_0's l2: 4.63631	valid_0's wrmsse: 0.548395
[239]	training's l2: 3.51876	training's wrmsse: 0.411324	valid_0's l2: 4.63681	valid_0's wrmsse: 0.54837
[240]	training's l2: 3.51778	training's wrmsse: 0.411239	valid_0's l2: 4.63688	valid_0's wrmsse: 0.548508
[241]	training's l2: 3.51653	training's wrmsse: 0.411017	valid_0's l2: 4.63727	valid_0's wrmsse: 0.548552
[242]	training's l2: 3.51452	training's wrmsse: 0.410652	valid_0's l2: 4.63691	valid_0's wrmsse: 0.548666
[243]	training's l2: 3.51228	training's wrmsse: 0.410464	valid_0's l2: 4.63674	valid_0's wrmsse: 0.549021
[244]	training's l2: 3.51054	training's wrmsse: 0.410074	valid_0's l2: 4.63642	valid_0's wrmsse: 0.548353
[245]	training's l2: 3.50828	training's wrmsse: 0

In [35]:
# weight_scale 1e4
params = {
    'boosting_type': 'gbdt',
    'num_leaves': 1024,
    'learning_rate': 0.1,
    'metrics':'l2',
    #'feature_fraction': 0.9,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    'verbose': 1
}

gbm = lgb.train(params, train_dset, valid_sets=[val_dset, train_dset], feature_name=col_names, categorical_feature=[0,1,2,3,4],
                fobj=custom_obj, feval=custom_eval, num_boost_round=300)

[1]	training's l2: 12.9092	training's wrmsse: 4.25589	valid_0's l2: 13.7885	valid_0's wrmsse: 4.85393
[2]	training's l2: 11.5046	training's wrmsse: 3.86544	valid_0's l2: 12.1383	valid_0's wrmsse: 4.41085
[3]	training's l2: 10.3593	training's wrmsse: 3.51514	valid_0's l2: 10.7961	valid_0's wrmsse: 4.01142
[4]	training's l2: 9.42045	training's wrmsse: 3.20035	valid_0's l2: 9.70289	valid_0's wrmsse: 3.65313
[5]	training's l2: 8.65427	training's wrmsse: 2.91781	valid_0's l2: 8.83416	valid_0's wrmsse: 3.33176
[6]	training's l2: 8.02086	training's wrmsse: 2.66449	valid_0's l2: 8.10937	valid_0's wrmsse: 3.04171
[7]	training's l2: 7.50109	training's wrmsse: 2.43718	valid_0's l2: 7.51524	valid_0's wrmsse: 2.78193
[8]	training's l2: 7.07377	training's wrmsse: 2.2331	valid_0's l2: 7.0233	valid_0's wrmsse: 2.54706
[9]	training's l2: 6.71416	training's wrmsse: 2.05095	valid_0's l2: 6.63172	valid_0's wrmsse: 2.33854
[10]	training's l2: 6.42087	training's wrmsse: 1.88724	valid_0's l2: 6.30225	valid_0

[80]	training's l2: 3.87117	training's wrmsse: 0.471872	valid_0's l2: 4.69925	valid_0's wrmsse: 0.57666
[81]	training's l2: 3.86162	training's wrmsse: 0.47074	valid_0's l2: 4.69969	valid_0's wrmsse: 0.573912
[82]	training's l2: 3.8548	training's wrmsse: 0.46849	valid_0's l2: 4.69911	valid_0's wrmsse: 0.57329
[83]	training's l2: 3.84615	training's wrmsse: 0.467179	valid_0's l2: 4.69813	valid_0's wrmsse: 0.572612
[84]	training's l2: 3.83822	training's wrmsse: 0.466243	valid_0's l2: 4.69831	valid_0's wrmsse: 0.574574
[85]	training's l2: 3.83155	training's wrmsse: 0.462333	valid_0's l2: 4.69887	valid_0's wrmsse: 0.573969
[86]	training's l2: 3.82461	training's wrmsse: 0.461481	valid_0's l2: 4.69716	valid_0's wrmsse: 0.57151
[87]	training's l2: 3.81592	training's wrmsse: 0.460456	valid_0's l2: 4.69415	valid_0's wrmsse: 0.571051
[88]	training's l2: 3.80765	training's wrmsse: 0.459096	valid_0's l2: 4.6927	valid_0's wrmsse: 0.571129
[89]	training's l2: 3.79825	training's wrmsse: 0.457676	valid_

[158]	training's l2: 3.41099	training's wrmsse: 0.402672	valid_0's l2: 4.65931	valid_0's wrmsse: 0.553481
[159]	training's l2: 3.40756	training's wrmsse: 0.402453	valid_0's l2: 4.66035	valid_0's wrmsse: 0.553524
[160]	training's l2: 3.40352	training's wrmsse: 0.402207	valid_0's l2: 4.65981	valid_0's wrmsse: 0.553512
[161]	training's l2: 3.39898	training's wrmsse: 0.401672	valid_0's l2: 4.6593	valid_0's wrmsse: 0.553397
[162]	training's l2: 3.39538	training's wrmsse: 0.401341	valid_0's l2: 4.65891	valid_0's wrmsse: 0.553601
[163]	training's l2: 3.39158	training's wrmsse: 0.400786	valid_0's l2: 4.65863	valid_0's wrmsse: 0.553121
[164]	training's l2: 3.38673	training's wrmsse: 0.400447	valid_0's l2: 4.65914	valid_0's wrmsse: 0.553179
[165]	training's l2: 3.38373	training's wrmsse: 0.400042	valid_0's l2: 4.65816	valid_0's wrmsse: 0.553154
[166]	training's l2: 3.38092	training's wrmsse: 0.399793	valid_0's l2: 4.65805	valid_0's wrmsse: 0.553317
[167]	training's l2: 3.37691	training's wrmsse:

[236]	training's l2: 3.15617	training's wrmsse: 0.372645	valid_0's l2: 4.6539	valid_0's wrmsse: 0.550066
[237]	training's l2: 3.15466	training's wrmsse: 0.37253	valid_0's l2: 4.65427	valid_0's wrmsse: 0.550165
[238]	training's l2: 3.15324	training's wrmsse: 0.372453	valid_0's l2: 4.65419	valid_0's wrmsse: 0.550106
[239]	training's l2: 3.152	training's wrmsse: 0.37235	valid_0's l2: 4.65423	valid_0's wrmsse: 0.549988
[240]	training's l2: 3.15075	training's wrmsse: 0.372273	valid_0's l2: 4.65432	valid_0's wrmsse: 0.550034
[241]	training's l2: 3.1484	training's wrmsse: 0.371729	valid_0's l2: 4.65416	valid_0's wrmsse: 0.5502
[242]	training's l2: 3.14494	training's wrmsse: 0.371317	valid_0's l2: 4.65386	valid_0's wrmsse: 0.550422
[243]	training's l2: 3.14127	training's wrmsse: 0.37098	valid_0's l2: 4.65364	valid_0's wrmsse: 0.549936
[244]	training's l2: 3.13926	training's wrmsse: 0.370777	valid_0's l2: 4.65424	valid_0's wrmsse: 0.550121
[245]	training's l2: 3.1377	training's wrmsse: 0.370649

In [37]:
# weight_scale 1e2
params = {
    'boosting_type': 'gbdt',
    'num_leaves': 512,
    'learning_rate': 0.1,
    'metrics':'l2',
    #'feature_fraction': 0.9,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    'verbose': 1
}

gbm = lgb.train(params, train_dset, valid_sets=[val_dset, train_dset], feature_name=col_names, categorical_feature=[0,1,2,3,4],
                fobj=custom_obj, feval=custom_eval, num_boost_round=500)

[1]	training's l2: 13.0537	training's wrmsse: 4.25795	valid_0's l2: 13.901	valid_0's wrmsse: 4.85652
[2]	training's l2: 11.7711	training's wrmsse: 3.86916	valid_0's l2: 12.3591	valid_0's wrmsse: 4.41602
[3]	training's l2: 10.7222	training's wrmsse: 3.52035	valid_0's l2: 11.0904	valid_0's wrmsse: 4.01933
[4]	training's l2: 9.85949	training's wrmsse: 3.20686	valid_0's l2: 10.037	valid_0's wrmsse: 3.66215
[5]	training's l2: 9.14991	training's wrmsse: 2.92553	valid_0's l2: 9.17814	valid_0's wrmsse: 3.34378
[6]	training's l2: 8.56762	training's wrmsse: 2.67317	valid_0's l2: 8.46188	valid_0's wrmsse: 3.05553
[7]	training's l2: 8.09751	training's wrmsse: 2.44725	valid_0's l2: 7.8847	valid_0's wrmsse: 2.7958
[8]	training's l2: 7.70733	training's wrmsse: 2.24485	valid_0's l2: 7.40058	valid_0's wrmsse: 2.56269
[9]	training's l2: 7.38717	training's wrmsse: 2.06343	valid_0's l2: 7.00611	valid_0's wrmsse: 2.35192
[10]	training's l2: 7.12251	training's wrmsse: 1.90143	valid_0's l2: 6.6746	valid_0's 

[80]	training's l2: 5.476	training's wrmsse: 0.519192	valid_0's l2: 4.93243	valid_0's wrmsse: 0.561348
[81]	training's l2: 5.46976	training's wrmsse: 0.518645	valid_0's l2: 4.93232	valid_0's wrmsse: 0.561607
[82]	training's l2: 5.46637	training's wrmsse: 0.515984	valid_0's l2: 4.93145	valid_0's wrmsse: 0.55994
[83]	training's l2: 5.46184	training's wrmsse: 0.515314	valid_0's l2: 4.93031	valid_0's wrmsse: 0.55956
[84]	training's l2: 5.45911	training's wrmsse: 0.513477	valid_0's l2: 4.92808	valid_0's wrmsse: 0.555966
[85]	training's l2: 5.45417	training's wrmsse: 0.512944	valid_0's l2: 4.92623	valid_0's wrmsse: 0.555881
[86]	training's l2: 5.4517	training's wrmsse: 0.511055	valid_0's l2: 4.92538	valid_0's wrmsse: 0.554895
[87]	training's l2: 5.44561	training's wrmsse: 0.509985	valid_0's l2: 4.92396	valid_0's wrmsse: 0.554017
[88]	training's l2: 5.44255	training's wrmsse: 0.509226	valid_0's l2: 4.92237	valid_0's wrmsse: 0.553925
[89]	training's l2: 5.44013	training's wrmsse: 0.508148	vali

[158]	training's l2: 5.25409	training's wrmsse: 0.469429	valid_0's l2: 4.89009	valid_0's wrmsse: 0.542781
[159]	training's l2: 5.25216	training's wrmsse: 0.4693	valid_0's l2: 4.88961	valid_0's wrmsse: 0.542347
[160]	training's l2: 5.25096	training's wrmsse: 0.468139	valid_0's l2: 4.88944	valid_0's wrmsse: 0.542199
[161]	training's l2: 5.2476	training's wrmsse: 0.467834	valid_0's l2: 4.88961	valid_0's wrmsse: 0.542182
[162]	training's l2: 5.24588	training's wrmsse: 0.467623	valid_0's l2: 4.88946	valid_0's wrmsse: 0.541933
[163]	training's l2: 5.24456	training's wrmsse: 0.467299	valid_0's l2: 4.8891	valid_0's wrmsse: 0.543043
[164]	training's l2: 5.24246	training's wrmsse: 0.466815	valid_0's l2: 4.88916	valid_0's wrmsse: 0.543069
[165]	training's l2: 5.24078	training's wrmsse: 0.466409	valid_0's l2: 4.8885	valid_0's wrmsse: 0.543283
[166]	training's l2: 5.23968	training's wrmsse: 0.465868	valid_0's l2: 4.88791	valid_0's wrmsse: 0.542886
[167]	training's l2: 5.23812	training's wrmsse: 0.4

[236]	training's l2: 5.13417	training's wrmsse: 0.445047	valid_0's l2: 4.87081	valid_0's wrmsse: 0.540839
[237]	training's l2: 5.13343	training's wrmsse: 0.444754	valid_0's l2: 4.87082	valid_0's wrmsse: 0.540718
[238]	training's l2: 5.13235	training's wrmsse: 0.444493	valid_0's l2: 4.87053	valid_0's wrmsse: 0.540704
[239]	training's l2: 5.13073	training's wrmsse: 0.4443	valid_0's l2: 4.87031	valid_0's wrmsse: 0.5409
[240]	training's l2: 5.12973	training's wrmsse: 0.44422	valid_0's l2: 4.87066	valid_0's wrmsse: 0.540977
[241]	training's l2: 5.1281	training's wrmsse: 0.444022	valid_0's l2: 4.87048	valid_0's wrmsse: 0.540878
[242]	training's l2: 5.12742	training's wrmsse: 0.443561	valid_0's l2: 4.87073	valid_0's wrmsse: 0.54132
[243]	training's l2: 5.12608	training's wrmsse: 0.44346	valid_0's l2: 4.87041	valid_0's wrmsse: 0.541155
[244]	training's l2: 5.12454	training's wrmsse: 0.443296	valid_0's l2: 4.87119	valid_0's wrmsse: 0.541043
[245]	training's l2: 5.12336	training's wrmsse: 0.4432

[314]	training's l2: 5.04465	training's wrmsse: 0.430532	valid_0's l2: 4.86064	valid_0's wrmsse: 0.537974
[315]	training's l2: 5.04402	training's wrmsse: 0.43046	valid_0's l2: 4.86051	valid_0's wrmsse: 0.538075
[316]	training's l2: 5.04228	training's wrmsse: 0.429795	valid_0's l2: 4.86051	valid_0's wrmsse: 0.538128
[317]	training's l2: 5.04178	training's wrmsse: 0.429617	valid_0's l2: 4.8605	valid_0's wrmsse: 0.538821
[318]	training's l2: 5.04105	training's wrmsse: 0.429539	valid_0's l2: 4.86076	valid_0's wrmsse: 0.538747
[319]	training's l2: 5.03992	training's wrmsse: 0.429495	valid_0's l2: 4.86143	valid_0's wrmsse: 0.538682
[320]	training's l2: 5.03925	training's wrmsse: 0.429405	valid_0's l2: 4.86111	valid_0's wrmsse: 0.538259
[321]	training's l2: 5.03811	training's wrmsse: 0.429198	valid_0's l2: 4.86113	valid_0's wrmsse: 0.538286
[322]	training's l2: 5.03689	training's wrmsse: 0.429079	valid_0's l2: 4.86092	valid_0's wrmsse: 0.538411
[323]	training's l2: 5.0363	training's wrmsse: 0

[392]	training's l2: 4.97233	training's wrmsse: 0.419075	valid_0's l2: 4.85666	valid_0's wrmsse: 0.536449
[393]	training's l2: 4.97196	training's wrmsse: 0.419024	valid_0's l2: 4.85677	valid_0's wrmsse: 0.5364
[394]	training's l2: 4.97122	training's wrmsse: 0.418955	valid_0's l2: 4.85649	valid_0's wrmsse: 0.536299
[395]	training's l2: 4.97052	training's wrmsse: 0.418546	valid_0's l2: 4.85631	valid_0's wrmsse: 0.535616
[396]	training's l2: 4.96856	training's wrmsse: 0.4185	valid_0's l2: 4.85691	valid_0's wrmsse: 0.535612
[397]	training's l2: 4.96669	training's wrmsse: 0.418377	valid_0's l2: 4.85589	valid_0's wrmsse: 0.535778
[398]	training's l2: 4.96554	training's wrmsse: 0.418114	valid_0's l2: 4.85582	valid_0's wrmsse: 0.535801
[399]	training's l2: 4.96468	training's wrmsse: 0.418094	valid_0's l2: 4.85587	valid_0's wrmsse: 0.535888
[400]	training's l2: 4.96377	training's wrmsse: 0.417935	valid_0's l2: 4.85595	valid_0's wrmsse: 0.535869
[401]	training's l2: 4.96156	training's wrmsse: 0.

[470]	training's l2: 4.90299	training's wrmsse: 0.409549	valid_0's l2: 4.84994	valid_0's wrmsse: 0.532927
[471]	training's l2: 4.90226	training's wrmsse: 0.409432	valid_0's l2: 4.85015	valid_0's wrmsse: 0.533015
[472]	training's l2: 4.90179	training's wrmsse: 0.409393	valid_0's l2: 4.85037	valid_0's wrmsse: 0.532991
[473]	training's l2: 4.90122	training's wrmsse: 0.409386	valid_0's l2: 4.85068	valid_0's wrmsse: 0.532847
[474]	training's l2: 4.89959	training's wrmsse: 0.409337	valid_0's l2: 4.85083	valid_0's wrmsse: 0.533102
[475]	training's l2: 4.89911	training's wrmsse: 0.409349	valid_0's l2: 4.85096	valid_0's wrmsse: 0.533155
[476]	training's l2: 4.89809	training's wrmsse: 0.409365	valid_0's l2: 4.85106	valid_0's wrmsse: 0.533155
[477]	training's l2: 4.89735	training's wrmsse: 0.409345	valid_0's l2: 4.85123	valid_0's wrmsse: 0.533156
[478]	training's l2: 4.89614	training's wrmsse: 0.409107	valid_0's l2: 4.85097	valid_0's wrmsse: 0.533313
[479]	training's l2: 4.89532	training's wrmsse

In [ ]:
# weight_scale 1e2
params = {
    'boosting_type': 'gbdt',
    'num_leaves': 1024,
    'learning_rate': 0.1,
    'metrics':'l2',
    #'feature_fraction': 0.9,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    'verbose': 1
}

gbm = lgb.train(params, train_dset, valid_sets=[val_dset, train_dset], feature_name=col_names, categorical_feature=[0,1,2,3,4],
                fobj=custom_obj, feval=custom_eval, num_boost_round=500)

[1]	training's l2: 13.0516	training's wrmsse: 4.25618	valid_0's l2: 13.8978	valid_0's wrmsse: 4.85354
[2]	training's l2: 11.7675	training's wrmsse: 3.86593	valid_0's l2: 12.3537	valid_0's wrmsse: 4.41186
[3]	training's l2: 10.7172	training's wrmsse: 3.51564	valid_0's l2: 11.083	valid_0's wrmsse: 4.0136
[4]	training's l2: 9.85356	training's wrmsse: 3.20111	valid_0's l2: 10.03	valid_0's wrmsse: 3.65426
[5]	training's l2: 9.14858	training's wrmsse: 2.91859	valid_0's l2: 9.1694	valid_0's wrmsse: 3.3316
[6]	training's l2: 8.56573	training's wrmsse: 2.66503	valid_0's l2: 8.45059	valid_0's wrmsse: 3.03981
[7]	training's l2: 8.09638	training's wrmsse: 2.43803	valid_0's l2: 7.8837	valid_0's wrmsse: 2.77899
[8]	training's l2: 7.70459	training's wrmsse: 2.23446	valid_0's l2: 7.39808	valid_0's wrmsse: 2.54379
[9]	training's l2: 7.38313	training's wrmsse: 2.05201	valid_0's l2: 7.00402	valid_0's wrmsse: 2.33166
[10]	training's l2: 7.11767	training's wrmsse: 1.88866	valid_0's l2: 6.67219	valid_0's wr

[80]	training's l2: 5.45844	training's wrmsse: 0.500958	valid_0's l2: 4.93643	valid_0's wrmsse: 0.563259
[81]	training's l2: 5.45451	training's wrmsse: 0.499249	valid_0's l2: 4.9354	valid_0's wrmsse: 0.563286
[82]	training's l2: 5.44843	training's wrmsse: 0.498688	valid_0's l2: 4.93308	valid_0's wrmsse: 0.563215
[83]	training's l2: 5.44232	training's wrmsse: 0.497982	valid_0's l2: 4.9322	valid_0's wrmsse: 0.562829
[84]	training's l2: 5.43883	training's wrmsse: 0.496418	valid_0's l2: 4.92985	valid_0's wrmsse: 0.561435
[85]	training's l2: 5.43388	training's wrmsse: 0.494977	valid_0's l2: 4.92665	valid_0's wrmsse: 0.560272
[86]	training's l2: 5.42722	training's wrmsse: 0.493726	valid_0's l2: 4.92692	valid_0's wrmsse: 0.560127
[87]	training's l2: 5.42035	training's wrmsse: 0.493291	valid_0's l2: 4.92527	valid_0's wrmsse: 0.560204
[88]	training's l2: 5.4176	training's wrmsse: 0.491907	valid_0's l2: 4.92403	valid_0's wrmsse: 0.558426
[89]	training's l2: 5.41529	training's wrmsse: 0.490591	va

[158]	training's l2: 5.20776	training's wrmsse: 0.450331	valid_0's l2: 4.89041	valid_0's wrmsse: 0.549464
[159]	training's l2: 5.20436	training's wrmsse: 0.450091	valid_0's l2: 4.89008	valid_0's wrmsse: 0.549272
[160]	training's l2: 5.20243	training's wrmsse: 0.449833	valid_0's l2: 4.89025	valid_0's wrmsse: 0.54938
[161]	training's l2: 5.19968	training's wrmsse: 0.449628	valid_0's l2: 4.88978	valid_0's wrmsse: 0.549136
[162]	training's l2: 5.19814	training's wrmsse: 0.449041	valid_0's l2: 4.89003	valid_0's wrmsse: 0.54873
[163]	training's l2: 5.19569	training's wrmsse: 0.448874	valid_0's l2: 4.88949	valid_0's wrmsse: 0.548585
[164]	training's l2: 5.1933	training's wrmsse: 0.44818	valid_0's l2: 4.88941	valid_0's wrmsse: 0.548668
[165]	training's l2: 5.19041	training's wrmsse: 0.44788	valid_0's l2: 4.88826	valid_0's wrmsse: 0.549098
[166]	training's l2: 5.18848	training's wrmsse: 0.447734	valid_0's l2: 4.88771	valid_0's wrmsse: 0.54863
[167]	training's l2: 5.18643	training's wrmsse: 0.44

[236]	training's l2: 5.06149	training's wrmsse: 0.428547	valid_0's l2: 4.87114	valid_0's wrmsse: 0.545285
[237]	training's l2: 5.0602	training's wrmsse: 0.428072	valid_0's l2: 4.87095	valid_0's wrmsse: 0.545338
[238]	training's l2: 5.05874	training's wrmsse: 0.427772	valid_0's l2: 4.87115	valid_0's wrmsse: 0.545377
[239]	training's l2: 5.05712	training's wrmsse: 0.427673	valid_0's l2: 4.87113	valid_0's wrmsse: 0.545414
[240]	training's l2: 5.05645	training's wrmsse: 0.42748	valid_0's l2: 4.87077	valid_0's wrmsse: 0.544781
[241]	training's l2: 5.05529	training's wrmsse: 0.427358	valid_0's l2: 4.87011	valid_0's wrmsse: 0.545282
[242]	training's l2: 5.05426	training's wrmsse: 0.427245	valid_0's l2: 4.86981	valid_0's wrmsse: 0.54535
[243]	training's l2: 5.05125	training's wrmsse: 0.427032	valid_0's l2: 4.86934	valid_0's wrmsse: 0.545099
[244]	training's l2: 5.05043	training's wrmsse: 0.42684	valid_0's l2: 4.86937	valid_0's wrmsse: 0.545017
[245]	training's l2: 5.0491	training's wrmsse: 0.4

[314]	training's l2: 4.96696	training's wrmsse: 0.41371	valid_0's l2: 4.86017	valid_0's wrmsse: 0.544511
[315]	training's l2: 4.96568	training's wrmsse: 0.413608	valid_0's l2: 4.85991	valid_0's wrmsse: 0.544372
[316]	training's l2: 4.96485	training's wrmsse: 0.41355	valid_0's l2: 4.85952	valid_0's wrmsse: 0.544342
[317]	training's l2: 4.96397	training's wrmsse: 0.413478	valid_0's l2: 4.85927	valid_0's wrmsse: 0.544484
[318]	training's l2: 4.96284	training's wrmsse: 0.413315	valid_0's l2: 4.85961	valid_0's wrmsse: 0.544427
[319]	training's l2: 4.96177	training's wrmsse: 0.41324	valid_0's l2: 4.85983	valid_0's wrmsse: 0.544265
[320]	training's l2: 4.96067	training's wrmsse: 0.413197	valid_0's l2: 4.85994	valid_0's wrmsse: 0.544277
[321]	training's l2: 4.95956	training's wrmsse: 0.413104	valid_0's l2: 4.85967	valid_0's wrmsse: 0.544386
[322]	training's l2: 4.95867	training's wrmsse: 0.412818	valid_0's l2: 4.85984	valid_0's wrmsse: 0.544564
[323]	training's l2: 4.95783	training's wrmsse: 0

[392]	training's l2: 4.88285	training's wrmsse: 0.402912	valid_0's l2: 4.8531	valid_0's wrmsse: 0.542667
[393]	training's l2: 4.88164	training's wrmsse: 0.402558	valid_0's l2: 4.85301	valid_0's wrmsse: 0.542886
[394]	training's l2: 4.88059	training's wrmsse: 0.402506	valid_0's l2: 4.85292	valid_0's wrmsse: 0.542733
[395]	training's l2: 4.87987	training's wrmsse: 0.402259	valid_0's l2: 4.85238	valid_0's wrmsse: 0.542776
[396]	training's l2: 4.87911	training's wrmsse: 0.402143	valid_0's l2: 4.8522	valid_0's wrmsse: 0.542663
[397]	training's l2: 4.8782	training's wrmsse: 0.40205	valid_0's l2: 4.85223	valid_0's wrmsse: 0.542448
[398]	training's l2: 4.87757	training's wrmsse: 0.401968	valid_0's l2: 4.85267	valid_0's wrmsse: 0.542348
[399]	training's l2: 4.87678	training's wrmsse: 0.401729	valid_0's l2: 4.85261	valid_0's wrmsse: 0.541815
[400]	training's l2: 4.87604	training's wrmsse: 0.401637	valid_0's l2: 4.85281	valid_0's wrmsse: 0.541782
[401]	training's l2: 4.87537	training's wrmsse: 0.

In [31]:
gc.collect()

230

In [32]:
sorted(list(zip(col_names, gbm.feature_importance())), key=lambda item: item[1], reverse=True)

[('sell_price', 16000),
 ('_28dl_28dm', 10207),
 ('_28dl_7dm', 9305),
 ('_28dl_7ds', 7628),
 ('_28dl_14dm', 7592),
 ('_28dl_21dm', 6912),
 ('_28dl_28ds', 6859),
 ('_28dl', 6756),
 ('_28dl_14ds', 5671),
 ('day_id', 5648),
 ('_28dl_21ds', 5180),
 ('day_of_week', 4765),
 ('state_id', 4740),
 ('item_id', 4465),
 ('day_of_month', 4208),
 ('LaborDay_lag', 2599),
 ('cat_id', 2513),
 ('Chanukah End_lag', 2435),
 ('SuperBowl_lag', 2362),
 ('StPatricksDay_lag', 2266),
 ('Thanksgiving_lag', 1915),
 ('Purim End_lag', 1903),
 ('Christmas_lag', 1696),
 ('ColumbusDay_lag', 1656),
 ('LentWeek2_lag', 1636),
 ('Easter_lag', 1609),
 ('NewYear_lag', 1506),
 ('IndependenceDay_lag', 1464),
 ('EidAlAdha_lag', 1458),
 ('Eid al-Fitr_lag', 1383),
 ('Pesach End_lag', 1366),
 ('Ramadan starts_lag', 1293),
 ('MartinLutherKingDay_lag', 1282),
 ('LentStart_lag', 1191),
 ('VeteransDay_lag', 1188),
 ('PresidentsDay_lag', 1181),
 ('Halloween_lag', 1170),
 ('OrthodoxChristmas_lag', 1108),
 ('NBAFinalsStart_lag', 1093),


In [33]:
gbm.save_model('gbm_customObj.txt')

In [34]:
! ls -alrh

total 18M
-rw-r--r-- 1 root root  18M Jun 30 05:49 gbm_customObj.txt
---------- 1 root root  263 Jun 30 04:52 __notebook_source__.ipynb
drwxr-xr-x 5 root root 4.0K Jun 30 04:52 ..
drwxr-xr-x 2 root root 4.0K Jun 30 05:49 .
